# Naive Bayes

The Naive Bayes series of classifiers are not really Bayesian, but the process kind of looks that way if you squint hard enough.  I like this classifier a lot because it's a fast, simple classifier which does reasonably well on datasets.  You can usually do better than Naive Bayes with a tailor-made model, but for something quick and dirty to see if you meet a minimum baseline for a go/no-go decision, it does well.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import CategoricalNB
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


Now let's prep the data.  Because we'll do it the same way for each, we only need to do this once.  I'll also remove the bits where we analyze the data, as we've seen it enough times already.

In [ ]:
campus_data = "../data/CampusRecruitment.csv"
df = pd.read_csv(campus_data, header=0)
y = df['status']
X = df.drop(['status', 'salary'], axis=1)

Something special about Naive Bayes is that we need to know the number of unique categories in each feature, and we have to do that before splitting into training and test data.  Otherwise, we can run into an error where a categorical value shows up for the first time in the test data.  We can use `nunique()` to get the number of unique values for each column, and then change it to an array for input.

In [ ]:
num_categories = X.nunique().array
clf = CategoricalNB(alpha=1, min_categories = num_categories)

## Pre-Processing

For this dataset, we want to use the columns leading up to `status` to determine if different college graduates were placed at a job.  Because the salary is determined by the placement status, we can't use it to predict if a new graduate will be placed, so we'll have to drop that column.  Note that if we were interested in doing a regression analysis, we could try to predict the salary given placement, but we're keeping it classy and sticking to classification algorithms only .

Unlike the heart attack dataset, this dataset includes non-numeric features.

In [ ]:
df

Before we can feed this data into the categorical Naive Bayes algorithm (or pretty much any other classification algorithm), we need to convert any text data into numeric data.  There are a few common techniques for encoding.  The technique we will use for our dataset is called ordinal encoding.  What it does is convert strings to ordinal values for encoding.  For example, `gender` has two values, M and F.  Ordinal encoding will set one of these to 0 and the other to 1.  We need to do this for each of the non-numeric features.

In [ ]:
enc = preprocessing.OrdinalEncoder()

X = enc.fit_transform(X)
X.shape

The `shape` here shows that we have the same number of rows as before (215) as well as the same number of columns (13).

In [ ]:
X

By contrast, I'm going to perform a simple label encoding of the `status`.

In [ ]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1740)

In [ ]:
X_train

Now let's train the model.

In [ ]:
clf = clf.fit(X_train, y_train)

## How'd We Do?

Let's first use the `accuracy_score` method in sklearn to see just how well we did.

In [ ]:
predicted = clf.predict(X_test)
accuracy_score(y_test, predicted)

Now let's review the confusion matrix and classification report.

In [ ]:
confusion_matrix(y_test, predicted)

In [ ]:
print(classification_report(y_test, predicted))

Compared to kNN, Naive Bayes did really well in indicating when a person would not be placed after graduation.  Unfortunately, it was rather pessimistic about our students, thinking that 31 of them would fail to be placed and missing on 14 of those predictions.  Its accuracy is a little worse than other classifiers we've seen, but in the same ballpark.